# 🚀 Model Deployment Tutorial

Welcome to the final tutorial in our ML Pipeline series! In this notebook, we'll deploy our trained models to production environments using modern deployment strategies and create production-ready APIs.

## 🎯 What You'll Learn
- Creating REST APIs for model serving
- Building web interfaces for predictions
- Docker containerization for consistent deployment
- Model monitoring and logging
- Production deployment strategies
- Health checks and error handling

## 🏆 Deployment Objectives
- **REST API**: Create production-ready model serving endpoints
- **Web Interface**: Build user-friendly prediction interface
- **Docker Container**: Package everything for consistent deployment
- **Monitoring**: Implement logging and health checks
- **Documentation**: Create API documentation and usage guides
- **Testing**: Implement comprehensive testing strategies

## 🛠️ Setup and Imports

In [ ]:
# =============================================================================
# UNIVERSAL SETUP - Works on all PCs and environments
# =============================================================================

import os
import sys
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Navigate to project root if we're in notebooks directory
if os.getcwd().endswith('notebooks'):
    os.chdir('..')
    print(f"📁 Changed to project root: {os.getcwd()}")
else:
    print(f"📁 Already in project root: {os.getcwd()}")

# Add src to Python path
src_path = os.path.join(os.getcwd(), 'src')
if src_path not in sys.path:
    sys.path.append(src_path)
    print(f"📦 Added to Python path: {src_path}")

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
from datetime import datetime
import json
import uuid
import logging
import time
import threading
from typing import Dict, List, Any, Optional

# Web framework imports
try:
    from flask import Flask, request, jsonify, render_template_string, send_from_directory
    from werkzeug.serving import make_server
    print("✅ Flask imported successfully")
    FLASK_AVAILABLE = True
except ImportError as e:
    print(f"⚠️ Flask not available: {e}")
    print("💡 Install with: pip install flask")
    FLASK_AVAILABLE = False

# MLflow imports (optional)
try:
    import mlflow
    import mlflow.sklearn
    MLFLOW_AVAILABLE = True
except ImportError:
    MLFLOW_AVAILABLE = False

# Scikit-learn imports
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn.metrics import accuracy_score, r2_score
from sklearn.model_selection import train_test_split

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger(__name__)

# Configure plotting
try:
    plt.style.use('seaborn-v0_8')
except:
    plt.style.use('seaborn')  # Fallback for older versions

sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (10, 6)

print("✅ Setup completed successfully!")
print(f"🌐 Flask available: {FLASK_AVAILABLE}")
print(f"🧪 MLflow available: {MLFLOW_AVAILABLE}")

## 📥 Load Trained Models

Let's load our best trained models from previous tutorials.

In [ ]:
class ModelLoader:
    """Load and manage trained models for deployment"""
    
    def __init__(self, models_dir="trained_models"):
        self.models_dir = Path(models_dir)
        self.loaded_models = {}
        self.model_metadata = {}
        self.feature_columns = {}
        
    def load_available_models(self):
        """Load all available trained models"""
        print("📥 Loading available trained models...")
        
        if not self.models_dir.exists():
            print(f"❌ Models directory not found: {self.models_dir}")
            print("💡 Please run the model training tutorial first")
            return False
        
        # Load model registry if available
        registry_file = self.models_dir / "model_registry.json"
        if registry_file.exists():
            try:
                with open(registry_file, 'r') as f:
                    registry = json.load(f)
                print(f"✅ Loaded model registry with {len(registry)} models")
                
                # Load models from registry
                for model_info in registry:
                    self._load_model_from_registry(model_info)
                    
            except Exception as e:
                print(f"⚠️ Error loading model registry: {e}")
        
        # Fallback: Load models directly from files
        if not self.loaded_models:
            self._load_models_from_files()
        
        # Load feature columns
        self._load_feature_columns()
        
        print(f"\n📊 Model Loading Summary:")
        print(f"   Total models loaded: {len(self.loaded_models)}")
        for model_name, model_info in self.loaded_models.items():
            print(f"   ✅ {model_name}: {model_info['algorithm']}")
        
        return len(self.loaded_models) > 0
    
    def _load_model_from_registry(self, model_info):
        """Load a model from registry information"""
        try:
            model_file = self.models_dir / model_info['filename']
            if model_file.exists():
                model = joblib.load(model_file)
                
                model_key = f"{model_info['dataset']}_{model_info['model_name'].lower().replace(' ', '_')}"
                
                self.loaded_models[model_key] = {
                    'model': model,
                    'algorithm': model_info['model_name'],
                    'dataset': model_info['dataset'],
                    'task_type': model_info['task_type'],
                    'performance': model_info.get('accuracy', model_info.get('r2_score', 0)),
                    'file_path': str(model_file)
                }
                
                self.model_metadata[model_key] = model_info
                
        except Exception as e:
            print(f"⚠️ Error loading model {model_info.get('filename', 'unknown')}: {e}")
    
    def _load_models_from_files(self):
        """Fallback: Load models directly from .joblib files"""
        print("🔄 Loading models directly from files...")
        
        model_files = list(self.models_dir.glob("*.joblib"))
        
        for model_file in model_files:
            try:
                model = joblib.load(model_file)
                
                # Parse filename to extract info
                filename = model_file.stem
                parts = filename.split('_')
                
                if len(parts) >= 2:
                    dataset = parts[0]
                    algorithm = '_'.join(parts[1:])
                    
                    # Determine task type
                    task_type = 'classification' if dataset == 'titanic' else 'regression'
                    
                    model_key = filename
                    
                    self.loaded_models[model_key] = {
                        'model': model,
                        'algorithm': algorithm.replace('_', ' ').title(),
                        'dataset': dataset,
                        'task_type': task_type,
                        'performance': 0.0,  # Unknown performance
                        'file_path': str(model_file)
                    }
                    
                    print(f"   ✅ Loaded {model_key}")
                    
            except Exception as e:
                print(f"   ⚠️ Error loading {model_file.name}: {e}")
    
    def _load_feature_columns(self):
        """Load feature column information"""
        # Try to load feature columns from data files
        feature_files = {
            'titanic': ['data/features/titanic_features.csv', 'data/raw/titanic.csv'],
            'housing': ['data/features/housing_features.csv', 'data/raw/housing.csv']
        }
        
        for dataset, paths in feature_files.items():
            for path in paths:
                if Path(path).exists():
                    try:
                        df = pd.read_csv(path)
                        target_col = 'Survived' if dataset == 'titanic' else 'MEDV'
                        
                        if target_col in df.columns:
                            feature_cols = [col for col in df.columns if col != target_col]
                            self.feature_columns[dataset] = feature_cols
                            print(f"   📊 Loaded {len(feature_cols)} feature columns for {dataset}")
                            break
                    except Exception as e:
                        continue
    
    def get_model(self, model_key):
        """Get a specific model"""
        return self.loaded_models.get(model_key)
    
    def get_best_model(self, dataset):
        """Get the best model for a dataset"""
        dataset_models = {k: v for k, v in self.loaded_models.items() 
                         if v['dataset'] == dataset}
        
        if not dataset_models:
            return None
        
        # Return model with highest performance
        best_key = max(dataset_models.keys(), 
                      key=lambda k: dataset_models[k]['performance'])
        return dataset_models[best_key]
    
    def list_models(self):
        """List all available models"""
        return list(self.loaded_models.keys())

# Initialize model loader
model_loader = ModelLoader()
models_loaded = model_loader.load_available_models()

if models_loaded:
    print("\n✅ Models loaded successfully for deployment!")
else:
    print("\n⚠️ No models available. Creating sample models for demonstration...")
    # Create sample models for demonstration
    from sklearn.datasets import make_classification, make_regression
    
    # Create sample data and models
    X_class, y_class = make_classification(n_samples=100, n_features=10, random_state=42)
    X_reg, y_reg = make_regression(n_samples=100, n_features=10, random_state=42)
    
    # Train sample models
    sample_classifier = LogisticRegression(random_state=42)
    sample_classifier.fit(X_class, y_class)
    
    sample_regressor = LinearRegression()
    sample_regressor.fit(X_reg, y_reg)
    
    # Add to model loader
    model_loader.loaded_models = {
        'titanic_sample_classifier': {
            'model': sample_classifier,
            'algorithm': 'Logistic Regression',
            'dataset': 'titanic',
            'task_type': 'classification',
            'performance': 0.85,
            'file_path': 'sample_model'
        },
        'housing_sample_regressor': {
            'model': sample_regressor,
            'algorithm': 'Linear Regression',
            'dataset': 'housing',
            'task_type': 'regression',
            'performance': 0.70,
            'file_path': 'sample_model'
        }
    }
    
    # Add sample feature columns
    model_loader.feature_columns = {
        'titanic': [f'feature_{i}' for i in range(10)],
        'housing': [f'feature_{i}' for i in range(10)]
    }
    
    print("✅ Sample models created for demonstration")

## 🌐 REST API Development

Let's create a production-ready REST API for model serving.

In [ ]:
class ModelServingAPI:
    """Production-ready model serving API"""
    
    def __init__(self, model_loader, host='0.0.0.0', port=5000):
        self.model_loader = model_loader
        self.host = host
        self.port = port
        self.app = None
        self.server = None
        self.prediction_count = 0
        self.start_time = datetime.now()
        
        if FLASK_AVAILABLE:
            self._create_flask_app()
        else:
            print("⚠️ Flask not available - API creation skipped")
    
    def _create_flask_app(self):
        """Create Flask application with all endpoints"""
        self.app = Flask(__name__)
        
        # Configure logging
        self.app.logger.setLevel(logging.INFO)
        
        # Register routes
        self._register_routes()
        
        print("✅ Flask API created successfully")
    
    def _register_routes(self):
        """Register all API routes"""
        
        @self.app.route('/', methods=['GET'])
        def home():
            """API home page with documentation"""
            return self._get_api_documentation()
        
        @self.app.route('/health', methods=['GET'])
        def health_check():
            """Health check endpoint"""
            uptime = datetime.now() - self.start_time
            return jsonify({
                'status': 'healthy',
                'uptime_seconds': int(uptime.total_seconds()),
                'models_loaded': len(self.model_loader.loaded_models),
                'predictions_served': self.prediction_count,
                'timestamp': datetime.now().isoformat()
            })
        
        @self.app.route('/models', methods=['GET'])
        def list_models():
            """List all available models"""
            models_info = []
            for model_key, model_info in self.model_loader.loaded_models.items():
                models_info.append({
                    'model_id': model_key,
                    'algorithm': model_info['algorithm'],
                    'dataset': model_info['dataset'],
                    'task_type': model_info['task_type'],
                    'performance': model_info['performance']
                })
            
            return jsonify({
                'models': models_info,
                'total_models': len(models_info)
            })
        
        @self.app.route('/predict/<model_id>', methods=['POST'])
        def predict(model_id):
            """Make predictions using specified model"""
            try:
                # Get model
                model_info = self.model_loader.get_model(model_id)
                if not model_info:
                    return jsonify({'error': f'Model {model_id} not found'}), 404
                
                # Get request data
                data = request.get_json()
                if not data:
                    return jsonify({'error': 'No JSON data provided'}), 400
                
                # Extract features
                if 'features' in data:
                    features = data['features']
                else:
                    return jsonify({'error': 'No features provided'}), 400
                
                # Convert to numpy array
                if isinstance(features, list):
                    if isinstance(features[0], list):
                        # Multiple samples
                        X = np.array(features)
                    else:
                        # Single sample
                        X = np.array([features])
                else:
                    return jsonify({'error': 'Features must be a list or list of lists'}), 400
                
                # Make prediction
                model = model_info['model']
                predictions = model.predict(X)
                
                # Get prediction probabilities if available
                probabilities = None
                if hasattr(model, 'predict_proba') and model_info['task_type'] == 'classification':
                    probabilities = model.predict_proba(X).tolist()
                
                # Increment prediction counter
                self.prediction_count += len(predictions)
                
                # Prepare response
                response = {
                    'model_id': model_id,
                    'algorithm': model_info['algorithm'],
                    'task_type': model_info['task_type'],
                    'predictions': predictions.tolist(),
                    'n_samples': len(predictions),
                    'timestamp': datetime.now().isoformat()
                }
                
                if probabilities:
                    response['probabilities'] = probabilities
                
                return jsonify(response)
                
            except Exception as e:
                self.app.logger.error(f"Prediction error: {str(e)}")
                return jsonify({'error': f'Prediction failed: {str(e)}'}), 500
        
        @self.app.route('/predict/titanic', methods=['POST'])
        def predict_titanic():
            """Simplified Titanic prediction endpoint"""
            try:
                # Get best Titanic model
                model_info = self.model_loader.get_best_model('titanic')
                if not model_info:
                    return jsonify({'error': 'No Titanic model available'}), 404
                
                # Get request data
                data = request.get_json()
                if not data:
                    return jsonify({'error': 'No JSON data provided'}), 400
                
                # Simple feature extraction for demo
                features = self._extract_titanic_features(data)
                
                # Make prediction
                model = model_info['model']
                prediction = model.predict([features])[0]
                
                # Get probability if available
                probability = None
                if hasattr(model, 'predict_proba'):
                    probability = model.predict_proba([features])[0][1]
                
                self.prediction_count += 1
                
                return jsonify({
                    'prediction': int(prediction),
                    'survival_probability': float(probability) if probability is not None else None,
                    'result': 'Survived' if prediction == 1 else 'Did not survive',
                    'model': model_info['algorithm'],
                    'timestamp': datetime.now().isoformat()
                })
                
            except Exception as e:
                self.app.logger.error(f"Titanic prediction error: {str(e)}")
                return jsonify({'error': f'Prediction failed: {str(e)}'}), 500
        
        @self.app.route('/predict/housing', methods=['POST'])
        def predict_housing():
            """Simplified Housing prediction endpoint"""
            try:
                # Get best Housing model
                model_info = self.model_loader.get_best_model('housing')
                if not model_info:
                    return jsonify({'error': 'No Housing model available'}), 404
                
                # Get request data
                data = request.get_json()
                if not data:
                    return jsonify({'error': 'No JSON data provided'}), 400
                
                # Simple feature extraction for demo
                features = self._extract_housing_features(data)
                
                # Make prediction
                model = model_info['model']
                prediction = model.predict([features])[0]
                
                self.prediction_count += 1
                
                return jsonify({
                    'predicted_price': float(prediction),
                    'price_range': self._get_price_category(prediction),
                    'model': model_info['algorithm'],
                    'timestamp': datetime.now().isoformat()
                })
                
            except Exception as e:
                self.app.logger.error(f"Housing prediction error: {str(e)}")
                return jsonify({'error': f'Prediction failed: {str(e)}'}), 500
        
        @self.app.route('/stats', methods=['GET'])
        def get_stats():
            """Get API usage statistics"""
            uptime = datetime.now() - self.start_time
            
            return jsonify({
                'uptime_seconds': int(uptime.total_seconds()),
                'uptime_formatted': str(uptime).split('.')[0],
                'total_predictions': self.prediction_count,
                'predictions_per_minute': round(self.prediction_count / max(uptime.total_seconds() / 60, 1), 2),
                'models_available': len(self.model_loader.loaded_models),
                'start_time': self.start_time.isoformat(),
                'current_time': datetime.now().isoformat()
            })
    
    def _extract_titanic_features(self, data):
        """Extract features for Titanic prediction (simplified)"""
        # This is a simplified version - in production, use the same feature engineering
        features = []
        
        # Use provided features or create dummy features
        if 'features' in data:
            return data['features']
        
        # Simple feature extraction from passenger data
        pclass = data.get('pclass', 3)
        sex = 1 if data.get('sex', 'male') == 'female' else 0
        age = data.get('age', 30)
        fare = data.get('fare', 15)
        
        # Create feature vector (simplified)
        features = [pclass, sex, age, fare, 0, 0, 1, 0, 0, 0]  # Pad to 10 features
        return features[:10]  # Ensure exactly 10 features
    
    def _extract_housing_features(self, data):
        """Extract features for Housing prediction (simplified)"""
        # This is a simplified version - in production, use the same feature engineering
        if 'features' in data:
            return data['features']
        
        # Simple feature extraction from house data
        crim = data.get('crime_rate', 0.1)
        rm = data.get('rooms', 6)
        age = data.get('age', 50)
        dis = data.get('distance', 5)
        
        # Create feature vector (simplified)
        features = [crim, 0, 0, 0, 0, rm, age, dis, 0, 0]  # Pad to 10 features
        return features[:10]  # Ensure exactly 10 features
    
    def _get_price_category(self, price):
        """Categorize house price"""
        if price < 15:
            return 'Low'
        elif price < 25:
            return 'Medium'
        elif price < 35:
            return 'High'
        else:
            return 'Very High'
    
    def _get_api_documentation(self):
        """Generate API documentation HTML"""
        html_template = """
        <!DOCTYPE html>
        <html>
        <head>
            <title>ML Model Serving API</title>
            <style>
                body { font-family: Arial, sans-serif; margin: 40px; background-color: #f5f5f5; }
                .container { max-width: 1000px; margin: 0 auto; background: white; padding: 30px; border-radius: 10px; box-shadow: 0 2px 10px rgba(0,0,0,0.1); }
                h1 { color: #333; text-align: center; }
                .endpoint { margin: 20px 0; padding: 15px; border: 1px solid #ddd; border-radius: 5px; }
                .method { display: inline-block; padding: 5px 10px; border-radius: 3px; color: white; font-weight: bold; }
                .get { background-color: #61affe; }
                .post { background-color: #49cc90; }
                code { background-color: #f4f4f4; padding: 2px 5px; border-radius: 3px; }
                pre { background-color: #f4f4f4; padding: 10px; border-radius: 5px; overflow-x: auto; }
            </style>
        </head>
        <body>
            <div class="container">
                <h1>🚀 ML Model Serving API</h1>
                <p>Production-ready API for machine learning model predictions</p>
                
                <h2>📊 Available Models</h2>
                <ul>
                    {% for model_key, model_info in models.items() %}
                    <li><strong>{{ model_key }}</strong>: {{ model_info.algorithm }} ({{ model_info.task_type }})</li>
                    {% endfor %}
                </ul>
                
                <h2>🌐 API Endpoints</h2>
                
                <div class="endpoint">
                    <h3><span class="method get">GET</span> /health</h3>
                    <p>Health check endpoint</p>
                    <pre>curl {{ base_url }}/health</pre>
                </div>
                
                <div class="endpoint">
                    <h3><span class="method get">GET</span> /models</h3>
                    <p>List all available models</p>
                    <pre>curl {{ base_url }}/models</pre>
                </div>
                
                <div class="endpoint">
                    <h3><span class="method post">POST</span> /predict/&lt;model_id&gt;</h3>
                    <p>Make predictions using specified model</p>
                    <pre>curl -X POST {{ base_url }}/predict/titanic_logistic_regression \
  -H "Content-Type: application/json" \
  -d '{"features": [3, 1, 22, 7.25, 1, 0, 0, 1, 0, 0]}'</pre>
                </div>
                
                <div class="endpoint">
                    <h3><span class="method post">POST</span> /predict/titanic</h3>
                    <p>Simplified Titanic survival prediction</p>
                    <pre>curl -X POST {{ base_url }}/predict/titanic \
  -H "Content-Type: application/json" \
  -d '{"pclass": 3, "sex": "female", "age": 25, "fare": 15}'</pre>
                </div>
                
                <div class="endpoint">
                    <h3><span class="method post">POST</span> /predict/housing</h3>
                    <p>Simplified housing price prediction</p>
                    <pre>curl -X POST {{ base_url }}/predict/housing \
  -H "Content-Type: application/json" \
  -d '{"crime_rate": 0.1, "rooms": 6, "age": 50, "distance": 5}'</pre>
                </div>
                
                <div class="endpoint">
                    <h3><span class="method get">GET</span> /stats</h3>
                    <p>Get API usage statistics</p>
                    <pre>curl {{ base_url }}/stats</pre>
                </div>
                
                <h2>📈 API Statistics</h2>
                <ul>
                    <li>Models loaded: {{ stats.models_loaded }}</li>
                    <li>Predictions served: {{ stats.predictions_served }}</li>
                    <li>Uptime: {{ stats.uptime }}</li>
                </ul>
            </div>
        </body>
        </html>
        """
        
        # Prepare template data
        uptime = datetime.now() - self.start_time
        base_url = f"http://{self.host}:{self.port}"
        
        template_data = {
            'models': self.model_loader.loaded_models,
            'base_url': base_url,
            'stats': {
                'models_loaded': len(self.model_loader.loaded_models),
                'predictions_served': self.prediction_count,
                'uptime': str(uptime).split('.')[0]
            }
        }
        
        # Simple template rendering (without Jinja2)
        html = html_template
        
        # Replace template variables
        models_list = ""
        for model_key, model_info in self.model_loader.loaded_models.items():
            models_list += f"<li><strong>{model_key}</strong>: {model_info['algorithm']} ({model_info['task_type']})</li>"
        
        html = html.replace('{% for model_key, model_info in models.items() %}', '')
        html = html.replace('{% endfor %}', models_list)
        html = html.replace('{{ base_url }}', base_url)
        html = html.replace('{{ stats.models_loaded }}', str(template_data['stats']['models_loaded']))
        html = html.replace('{{ stats.predictions_served }}', str(template_data['stats']['predictions_served']))
        html = html.replace('{{ stats.uptime }}', template_data['stats']['uptime'])
        
        return html
    
    def start_server(self, debug=False, threaded=True):
        """Start the API server"""
        if not FLASK_AVAILABLE:
            print("❌ Cannot start server - Flask not available")
            return False
        
        if not self.app:
            print("❌ Cannot start server - Flask app not created")
            return False
        
        try:
            print(f"🚀 Starting ML Model Serving API...")
            print(f"📍 Server URL: http://{self.host}:{self.port}")
            print(f"📚 API Documentation: http://{self.host}:{self.port}/")
            print(f"❤️ Health Check: http://{self.host}:{self.port}/health")
            print(f"🛑 Press Ctrl+C to stop the server")
            
            # Start server
            self.app.run(
                host=self.host,
                port=self.port,
                debug=debug,
                threaded=threaded
            )
            
        except Exception as e:
            print(f"❌ Error starting server: {e}")
            return False
        
        return True
    
    def start_server_background(self):
        """Start server in background thread"""
        if not FLASK_AVAILABLE or not self.app:
            print("❌ Cannot start background server")
            return False
        
        try:
            self.server = make_server(self.host, self.port, self.app, threaded=True)
            server_thread = threading.Thread(target=self.server.serve_forever)
            server_thread.daemon = True
            server_thread.start()
            
            print(f"🚀 ML Model Serving API started in background")
            print(f"📍 Server URL: http://{self.host}:{self.port}")
            
            return True
            
        except Exception as e:
            print(f"❌ Error starting background server: {e}")
            return False
    
    def stop_server(self):
        """Stop the background server"""
        if self.server:
            self.server.shutdown()
            print("🛑 Server stopped")

# Initialize API
api = ModelServingAPI(model_loader, host='localhost', port=5000)

if FLASK_AVAILABLE:
    print("✅ Model Serving API created successfully!")
    print(f"📊 API endpoints: {len(api.app.url_map._rules) if api.app else 0}")
else:
    print("⚠️ Model Serving API creation skipped (Flask not available)")

## 🌐 Web Interface Development

Let's create a user-friendly web interface for making predictions.

In [ ]:
def create_web_interface():
    """Create a comprehensive web interface for model predictions"""
    
    web_interface_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>🚀 ML Model Predictions</title>
    <style>
        * {
            margin: 0;
            padding: 0;
            box-sizing: border-box;
        }
        
        body {
            font-family: 'Segoe UI', Tahoma, Geneva, Verdana, sans-serif;
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            min-height: 100vh;
            padding: 20px;
        }
        
        .container {
            max-width: 1200px;
            margin: 0 auto;
            background: white;
            border-radius: 15px;
            box-shadow: 0 10px 30px rgba(0,0,0,0.2);
            overflow: hidden;
        }
        
        .header {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 30px;
            text-align: center;
        }
        
        .header h1 {
            font-size: 2.5em;
            margin-bottom: 10px;
        }
        
        .header p {
            font-size: 1.2em;
            opacity: 0.9;
        }
        
        .content {
            padding: 40px;
        }
        
        .model-section {
            margin-bottom: 40px;
            padding: 30px;
            border: 2px solid #e0e0e0;
            border-radius: 10px;
            transition: all 0.3s ease;
        }
        
        .model-section:hover {
            border-color: #667eea;
            box-shadow: 0 5px 15px rgba(102, 126, 234, 0.1);
        }
        
        .titanic-section {
            border-color: #3498db;
        }
        
        .housing-section {
            border-color: #e74c3c;
        }
        
        .section-title {
            font-size: 1.8em;
            margin-bottom: 20px;
            color: #333;
        }
        
        .form-group {
            margin-bottom: 20px;
        }
        
        .form-row {
            display: flex;
            gap: 20px;
            flex-wrap: wrap;
        }
        
        .form-col {
            flex: 1;
            min-width: 200px;
        }
        
        label {
            display: block;
            margin-bottom: 5px;
            font-weight: 600;
            color: #555;
        }
        
        input, select {
            width: 100%;
            padding: 12px;
            border: 2px solid #ddd;
            border-radius: 8px;
            font-size: 16px;
            transition: border-color 0.3s ease;
        }
        
        input:focus, select:focus {
            outline: none;
            border-color: #667eea;
        }
        
        .btn {
            background: linear-gradient(135deg, #667eea 0%, #764ba2 100%);
            color: white;
            padding: 15px 30px;
            border: none;
            border-radius: 8px;
            font-size: 16px;
            font-weight: 600;
            cursor: pointer;
            transition: all 0.3s ease;
            margin-top: 10px;
        }
        
        .btn:hover {
            transform: translateY(-2px);
            box-shadow: 0 5px 15px rgba(102, 126, 234, 0.3);
        }
        
        .result {
            margin-top: 20px;
            padding: 20px;
            border-radius: 8px;
            display: none;
        }
        
        .result.success {
            background-color: #d4edda;
            border: 1px solid #c3e6cb;
            color: #155724;
        }
        
        .result.error {
            background-color: #f8d7da;
            border: 1px solid #f5c6cb;
            color: #721c24;
        }
        
        .loading {
            display: none;
            text-align: center;
            margin-top: 20px;
        }
        
        .spinner {
            border: 4px solid #f3f3f3;
            border-top: 4px solid #667eea;
            border-radius: 50%;
            width: 40px;
            height: 40px;
            animation: spin 1s linear infinite;
            margin: 0 auto;
        }
        
        @keyframes spin {
            0% { transform: rotate(0deg); }
            100% { transform: rotate(360deg); }
        }
        
        .api-info {
            background-color: #f8f9fa;
            padding: 20px;
            border-radius: 8px;
            margin-top: 30px;
        }
        
        .api-info h3 {
            margin-bottom: 15px;
            color: #333;
        }
        
        .api-endpoint {
            background-color: #e9ecef;
            padding: 10px;
            border-radius: 5px;
            font-family: monospace;
            margin-bottom: 10px;
        }
        
        @media (max-width: 768px) {
            .form-row {
                flex-direction: column;
            }
            
            .header h1 {
                font-size: 2em;
            }
            
            .content {
                padding: 20px;
            }
        }
    </style>
</head>
<body>
    <div class="container">
        <div class="header">
            <h1>🚀 ML Model Predictions</h1>
            <p>Production-ready machine learning model serving interface</p>
        </div>
        
        <div class="content">
            <!-- Titanic Prediction Section -->
            <div class="model-section titanic-section">
                <h2 class="section-title">🚢 Titanic Survival Prediction</h2>
                <p>Predict passenger survival probability based on passenger characteristics</p>
                
                <form id="titanicForm">
                    <div class="form-row">
                        <div class="form-col">
                            <label for="pclass">Passenger Class:</label>
                            <select id="pclass" name="pclass">
                                <option value="1">First Class</option>
                                <option value="2">Second Class</option>
                                <option value="3" selected>Third Class</option>
                            </select>
                        </div>
                        
                        <div class="form-col">
                            <label for="sex">Gender:</label>
                            <select id="sex" name="sex">
                                <option value="male">Male</option>
                                <option value="female">Female</option>
                            </select>
                        </div>
                    </div>
                    
                    <div class="form-row">
                        <div class="form-col">
                            <label for="age">Age:</label>
                            <input type="number" id="age" name="age" value="30" min="0" max="100" step="1">
                        </div>
                        
                        <div class="form-col">
                            <label for="fare">Fare (£):</label>
                            <input type="number" id="fare" name="fare" value="15" min="0" step="0.01">
                        </div>
                    </div>
                    
                    <button type="button" class="btn" onclick="predictTitanic()">Predict Survival</button>
                </form>
                
                <div class="loading" id="titanicLoading">
                    <div class="spinner"></div>
                    <p>Making prediction...</p>
                </div>
                
                <div class="result" id="titanicResult"></div>
            </div>
            
            <!-- Housing Prediction Section -->
            <div class="model-section housing-section">
                <h2 class="section-title">🏠 Housing Price Prediction</h2>
                <p>Predict house price based on property characteristics</p>
                
                <form id="housingForm">
                    <div class="form-row">
                        <div class="form-col">
                            <label for="crime_rate">Crime Rate:</label>
                            <input type="number" id="crime_rate" name="crime_rate" value="0.1" min="0" step="0.01">
                        </div>
                        
                        <div class="form-col">
                            <label for="rooms">Average Rooms:</label>
                            <input type="number" id="rooms" name="rooms" value="6" min="1" max="10" step="0.1">
                        </div>
                    </div>
                    
                    <div class="form-row">
                        <div class="form-col">
                            <label for="house_age">Building Age (years):</label>
                            <input type="number" id="house_age" name="house_age" value="50" min="0" max="100" step="1">
                        </div>
                        
                        <div class="form-col">
                            <label for="distance">Distance to Employment Centers:</label>
                            <input type="number" id="distance" name="distance" value="5" min="0" step="0.1">
                        </div>
                    </div>
                    
                    <button type="button" class="btn" onclick="predictHousing()">Predict Price</button>
                </form>
                
                <div class="loading" id="housingLoading">
                    <div class="spinner"></div>
                    <p>Making prediction...</p>
                </div>
                
                <div class="result" id="housingResult"></div>
            </div>
            
            <!-- API Information -->
            <div class="api-info">
                <h3>🌐 API Endpoints</h3>
                <p>You can also use these endpoints programmatically:</p>
                
                <div class="api-endpoint">
                    POST /predict/titanic
                </div>
                
                <div class="api-endpoint">
                    POST /predict/housing
                </div>
                
                <div class="api-endpoint">
                    GET /health - Health check
                </div>
                
                <div class="api-endpoint">
                    GET /models - List available models
                </div>
            </div>
        </div>
    </div>
    
    <script>
        // API base URL - adjust if needed
        const API_BASE_URL = 'http://localhost:5000';
        
        async function predictTitanic() {
            const form = document.getElementById('titanicForm');
            const loading = document.getElementById('titanicLoading');
            const result = document.getElementById('titanicResult');
            
            // Show loading
            loading.style.display = 'block';
            result.style.display = 'none';
            
            try {
                // Get form data
                const formData = new FormData(form);
                const data = {
                    pclass: parseInt(formData.get('pclass')),
                    sex: formData.get('sex'),
                    age: parseFloat(formData.get('age')),
                    fare: parseFloat(formData.get('fare'))
                };
                
                // Make API call
                const response = await fetch(`${API_BASE_URL}/predict/titanic`, {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify(data)
                });
                
                if (!response.ok) {
                    throw new Error(`HTTP error! status: ${response.status}`);
                }
                
                const prediction = await response.json();
                
                // Display result
                result.className = 'result success';
                result.innerHTML = `
                    <h3>🎯 Prediction Result</h3>
                    <p><strong>Prediction:</strong> ${prediction.result}</p>
                    ${prediction.survival_probability ? `<p><strong>Survival Probability:</strong> ${(prediction.survival_probability * 100).toFixed(1)}%</p>` : ''}
                    <p><strong>Model:</strong> ${prediction.model}</p>
                    <p><strong>Timestamp:</strong> ${new Date(prediction.timestamp).toLocaleString()}</p>
                `;
                result.style.display = 'block';
                
            } catch (error) {
                console.error('Error:', error);
                result.className = 'result error';
                result.innerHTML = `
                    <h3>❌ Error</h3>
                    <p>Failed to make prediction. Please check if the API server is running.</p>
                    <p><strong>Error:</strong> ${error.message}</p>
                `;
                result.style.display = 'block';
            } finally {
                loading.style.display = 'none';
            }
        }
        
        async function predictHousing() {
            const form = document.getElementById('housingForm');
            const loading = document.getElementById('housingLoading');
            const result = document.getElementById('housingResult');
            
            // Show loading
            loading.style.display = 'block';
            result.style.display = 'none';
            
            try {
                // Get form data
                const formData = new FormData(form);
                const data = {
                    crime_rate: parseFloat(formData.get('crime_rate')),
                    rooms: parseFloat(formData.get('rooms')),
                    age: parseInt(formData.get('house_age')),
                    distance: parseFloat(formData.get('distance'))
                };
                
                // Make API call
                const response = await fetch(`${API_BASE_URL}/predict/housing`, {
                    method: 'POST',
                    headers: {
                        'Content-Type': 'application/json',
                    },
                    body: JSON.stringify(data)
                });
                
                if (!response.ok) {
                    throw new Error(`HTTP error! status: ${response.status}`);
                }
                
                const prediction = await response.json();
                
                // Display result
                result.className = 'result success';
                result.innerHTML = `
                    <h3>🎯 Prediction Result</h3>
                    <p><strong>Predicted Price:</strong> $${prediction.predicted_price.toFixed(1)}k</p>
                    <p><strong>Price Category:</strong> ${prediction.price_range}</p>
                    <p><strong>Model:</strong> ${prediction.model}</p>
                    <p><strong>Timestamp:</strong> ${new Date(prediction.timestamp).toLocaleString()}</p>
                `;
                result.style.display = 'block';
                
            } catch (error) {
                console.error('Error:', error);
                result.className = 'result error';
                result.innerHTML = `
                    <h3>❌ Error</h3>
                    <p>Failed to make prediction. Please check if the API server is running.</p>
                    <p><strong>Error:</strong> ${error.message}</p>
                `;
                result.style.display = 'block';
            } finally {
                loading.style.display = 'none';
            }
        }
        
        // Check API health on page load
        window.addEventListener('load', async function() {
            try {
                const response = await fetch(`${API_BASE_URL}/health`);
                if (response.ok) {
                    console.log('✅ API server is healthy');
                } else {
                    console.warn('⚠️ API server responded with error');
                }
            } catch (error) {
                console.warn('⚠️ API server not reachable:', error.message);
            }
        });
    </script>
</body>
</html>
    """
    
    # Save the web interface
    web_file = Path("web_interface.html")
    with open(web_file, 'w', encoding='utf-8') as f:
        f.write(web_interface_html)
    
    print(f"✅ Web interface created: {web_file}")
    print(f"🌐 Open in browser: file://{web_file.absolute()}")
    
    return web_file

# Create web interface
web_interface_file = create_web_interface()
print("\n🌐 Web interface ready for deployment!")

## 🐳 Docker Containerization

Let's create Docker configurations for consistent deployment.

In [ ]:
def create_docker_configurations():
    """Create Docker configurations for deployment"""
    print("🐳 Creating Docker configurations...")
    
    # Create deployment directory
    deploy_dir = Path("deployment")
    deploy_dir.mkdir(exist_ok=True)
    
    # 1. Dockerfile
    dockerfile_content = """
# Use Python 3.9 slim image
FROM python:3.9-slim

# Set working directory
WORKDIR /app

# Set environment variables
ENV PYTHONDONTWRITEBYTECODE=1
ENV PYTHONUNBUFFERED=1
ENV FLASK_APP=app.py
ENV FLASK_ENV=production

# Install system dependencies
RUN apt-get update && apt-get install -y \
    gcc \
    && rm -rf /var/lib/apt/lists/*

# Copy requirements first for better caching
COPY requirements.txt .

# Install Python dependencies
RUN pip install --no-cache-dir -r requirements.txt

# Copy application code
COPY . .

# Create non-root user
RUN useradd --create-home --shell /bin/bash app \
    && chown -R app:app /app
USER app

# Expose port
EXPOSE 5000

# Health check
HEALTHCHECK --interval=30s --timeout=30s --start-period=5s --retries=3 \
    CMD curl -f http://localhost:5000/health || exit 1

# Run the application
CMD ["python", "app.py"]
    """.strip()
    
    dockerfile_path = deploy_dir / "Dockerfile"
    with open(dockerfile_path, 'w') as f:
        f.write(dockerfile_content)
    
    # 2. Docker Compose
    docker_compose_content = """
version: '3.8'

services:
  ml-api:
    build:
      context: .
      dockerfile: Dockerfile
    ports:
      - "5000:5000"
    environment:
      - FLASK_ENV=production
      - MODEL_PATH=/app/models
    volumes:
      - ../trained_models:/app/models:ro
      - ./logs:/app/logs
    restart: unless-stopped
    healthcheck:
      test: ["CMD", "curl", "-f", "http://localhost:5000/health"]
      interval: 30s
      timeout: 10s
      retries: 3
      start_period: 40s
    networks:
      - ml-network

  nginx:
    image: nginx:alpine
    ports:
      - "80:80"
      - "443:443"
    volumes:
      - ./nginx.conf:/etc/nginx/nginx.conf:ro
      - ./web_interface.html:/usr/share/nginx/html/index.html:ro
    depends_on:
      - ml-api
    restart: unless-stopped
    networks:
      - ml-network

networks:
  ml-network:
    driver: bridge

volumes:
  model-data:
  log-data:
    """.strip()
    
    compose_path = deploy_dir / "docker-compose.yml"
    with open(compose_path, 'w') as f:
        f.write(docker_compose_content)
    
    # 3. Nginx Configuration
    nginx_config = """
events {
    worker_connections 1024;
}

http {
    upstream ml_api {
        server ml-api:5000;
    }
    
    server {
        listen 80;
        server_name localhost;
        
        # Serve static web interface
        location / {
            root /usr/share/nginx/html;
            index index.html;
        }
        
        # Proxy API requests
        location /api/ {
            proxy_pass http://ml_api/;
            proxy_set_header Host $host;
            proxy_set_header X-Real-IP $remote_addr;
            proxy_set_header X-Forwarded-For $proxy_add_x_forwarded_for;
            proxy_set_header X-Forwarded-Proto $scheme;
        }
        
        # Health check
        location /health {
            proxy_pass http://ml_api/health;
        }
    }
}
    """.strip()
    
    nginx_path = deploy_dir / "nginx.conf"
    with open(nginx_path, 'w') as f:
        f.write(nginx_config)
    
    # 4. Requirements for Docker
    docker_requirements = """
flask==2.3.3
scikit-learn==1.3.2
pandas==2.1.4
numpy==1.24.3
joblib==1.3.2
gunicorn==21.2.0
    """.strip()
    
    req_path = deploy_dir / "requirements.txt"
    with open(req_path, 'w') as f:
        f.write(docker_requirements)
    
    # 5. Production Flask App
    app_content = """
#!/usr/bin/env python3
"""
Production Flask application for ML model serving
"""

import os
import sys
from pathlib import Path
import logging
from flask import Flask, request, jsonify
import joblib
import numpy as np
from datetime import datetime

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s',
    handlers=[
        logging.FileHandler('logs/app.log'),
        logging.StreamHandler(sys.stdout)
    ]
)
logger = logging.getLogger(__name__)

# Create Flask app
app = Flask(__name__)

# Global variables
models = {}
prediction_count = 0
start_time = datetime.now()

def load_models():
    """Load all available models"""
    global models
    
    model_path = Path(os.getenv('MODEL_PATH', 'models'))
    if not model_path.exists():
        logger.warning(f"Model path {model_path} does not exist")
        return
    
    # Load models from directory
    for model_file in model_path.glob('*.joblib'):
        try:
            model = joblib.load(model_file)
            model_name = model_file.stem
            models[model_name] = model
            logger.info(f"Loaded model: {model_name}")
        except Exception as e:
            logger.error(f"Error loading model {model_file}: {e}")
    
    logger.info(f"Total models loaded: {len(models)}")

@app.route('/health', methods=['GET'])
def health_check():
    """Health check endpoint"""
    uptime = datetime.now() - start_time
    return jsonify({
        'status': 'healthy',
        'uptime_seconds': int(uptime.total_seconds()),
        'models_loaded': len(models),
        'predictions_served': prediction_count,
        'timestamp': datetime.now().isoformat()
    })

@app.route('/models', methods=['GET'])
def list_models():
    """List available models"""
    return jsonify({
        'models': list(models.keys()),
        'total_models': len(models)
    })

@app.route('/predict/<model_name>', methods=['POST'])
def predict(model_name):
    """Make prediction using specified model"""
    global prediction_count
    
    if model_name not in models:
        return jsonify({'error': f'Model {model_name} not found'}), 404
    
    try:
        data = request.get_json()
        if not data or 'features' not in data:
            return jsonify({'error': 'No features provided'}), 400
        
        features = np.array([data['features']])
        model = models[model_name]
        
        prediction = model.predict(features)[0]
        prediction_count += 1
        
        response = {
            'prediction': float(prediction),
            'model': model_name,
            'timestamp': datetime.now().isoformat()
        }
        
        # Add probability if available
        if hasattr(model, 'predict_proba'):
            proba = model.predict_proba(features)[0]
            response['probabilities'] = proba.tolist()
        
        logger.info(f"Prediction made with {model_name}: {prediction}")
        return jsonify(response)
        
    except Exception as e:
        logger.error(f"Prediction error: {e}")
        return jsonify({'error': str(e)}), 500

if __name__ == '__main__':
    # Create logs directory
    Path('logs').mkdir(exist_ok=True)
    
    # Load models
    load_models()
    
    # Start server
    port = int(os.getenv('PORT', 5000))
    app.run(host='0.0.0.0', port=port, debug=False)
    """.strip()
    
    app_path = deploy_dir / "app.py"
    with open(app_path, 'w') as f:
        f.write(app_content)
    
    # 6. Deployment Scripts
    deploy_script = """
#!/bin/bash
# Deployment script for ML Model API

echo "🚀 Starting ML Model API deployment..."

# Build and start containers
docker-compose down
docker-compose build --no-cache
docker-compose up -d

echo "✅ Deployment completed!"
echo "🌐 Web Interface: http://localhost"
echo "🔗 API Endpoint: http://localhost/api/health"
echo "📊 Check status: docker-compose ps"
echo "📝 View logs: docker-compose logs -f"
    """.strip()
    
    deploy_script_path = deploy_dir / "deploy.sh"
    with open(deploy_script_path, 'w') as f:
        f.write(deploy_script)
    
    # Make script executable
    try:
        deploy_script_path.chmod(0o755)
    except:
        pass  # Windows doesn't support chmod
    
    # 7. Copy web interface
    if Path("web_interface.html").exists():
        import shutil
        shutil.copy("web_interface.html", deploy_dir / "web_interface.html")
    
    print(f"✅ Docker configurations created in: {deploy_dir}")
    print(f"📁 Files created:")
    for file in deploy_dir.iterdir():
        print(f"   • {file.name}")
    
    return deploy_dir

# Create Docker configurations
docker_dir = create_docker_configurations()
print("\n🐳 Docker deployment ready!")
print(f"\n🚀 To deploy with Docker:")
print(f"   cd {docker_dir}")
print(f"   docker-compose up --build")

## 🧪 API Testing and Validation

Let's create comprehensive tests for our API.

In [ ]:
import requests
import time
import json

class APITester:
    """Comprehensive API testing suite"""
    
    def __init__(self, base_url="http://localhost:5000"):
        self.base_url = base_url
        self.test_results = []
    
    def test_health_endpoint(self):
        """Test health check endpoint"""
        print("🧪 Testing health endpoint...")
        
        try:
            response = requests.get(f"{self.base_url}/health", timeout=5)
            
            if response.status_code == 200:
                data = response.json()
                print(f"   ✅ Health check passed")
                print(f"   📊 Status: {data.get('status')}")
                print(f"   🤖 Models loaded: {data.get('models_loaded')}")
                print(f"   📈 Predictions served: {data.get('predictions_served')}")
                return True
            else:
                print(f"   ❌ Health check failed: {response.status_code}")
                return False
                
        except requests.exceptions.RequestException as e:
            print(f"   ❌ Health check failed: {e}")
            return False
    
    def test_models_endpoint(self):
        """Test models listing endpoint"""
        print("\n🧪 Testing models endpoint...")
        
        try:
            response = requests.get(f"{self.base_url}/models", timeout=5)
            
            if response.status_code == 200:
                data = response.json()
                print(f"   ✅ Models endpoint working")
                print(f"   📊 Total models: {data.get('total_models')}")
                
                if 'models' in data:
                    for model in data['models']:
                        if isinstance(model, dict):
                            print(f"   🤖 {model.get('model_id', 'Unknown')}: {model.get('algorithm', 'Unknown')}")
                        else:
                            print(f"   🤖 {model}")
                
                return True
            else:
                print(f"   ❌ Models endpoint failed: {response.status_code}")
                return False
                
        except requests.exceptions.RequestException as e:
            print(f"   ❌ Models endpoint failed: {e}")
            return False
    
    def test_titanic_prediction(self):
        """Test Titanic prediction endpoint"""
        print("\n🧪 Testing Titanic prediction...")
        
        test_cases = [
            {
                'name': 'First class female',
                'data': {'pclass': 1, 'sex': 'female', 'age': 25, 'fare': 50}
            },
            {
                'name': 'Third class male',
                'data': {'pclass': 3, 'sex': 'male', 'age': 30, 'fare': 10}
            }
        ]
        
        success_count = 0
        
        for test_case in test_cases:
            try:
                response = requests.post(
                    f"{self.base_url}/predict/titanic",
                    json=test_case['data'],
                    headers={'Content-Type': 'application/json'},
                    timeout=10
                )
                
                if response.status_code == 200:
                    data = response.json()
                    print(f"   ✅ {test_case['name']}: {data.get('result', 'Unknown')}")
                    if 'survival_probability' in data:
                        prob = data['survival_probability']
                        print(f"      📊 Survival probability: {prob:.1%}")
                    success_count += 1
                else:
                    print(f"   ❌ {test_case['name']}: HTTP {response.status_code}")
                    
            except requests.exceptions.RequestException as e:
                print(f"   ❌ {test_case['name']}: {e}")
        
        return success_count == len(test_cases)
    
    def test_housing_prediction(self):
        """Test Housing prediction endpoint"""
        print("\n🧪 Testing Housing prediction...")
        
        test_cases = [
            {
                'name': 'Low crime area',
                'data': {'crime_rate': 0.1, 'rooms': 7, 'age': 20, 'distance': 3}
            },
            {
                'name': 'High crime area',
                'data': {'crime_rate': 5.0, 'rooms': 5, 'age': 80, 'distance': 10}
            }
        ]
        
        success_count = 0
        
        for test_case in test_cases:
            try:
                response = requests.post(
                    f"{self.base_url}/predict/housing",
                    json=test_case['data'],
                    headers={'Content-Type': 'application/json'},
                    timeout=10
                )
                
                if response.status_code == 200:
                    data = response.json()
                    price = data.get('predicted_price', 0)
                    category = data.get('price_range', 'Unknown')
                    print(f"   ✅ {test_case['name']}: ${price:.1f}k ({category})")
                    success_count += 1
                else:
                    print(f"   ❌ {test_case['name']}: HTTP {response.status_code}")
                    
            except requests.exceptions.RequestException as e:
                print(f"   ❌ {test_case['name']}: {e}")
        
        return success_count == len(test_cases)
    
    def test_error_handling(self):
        """Test API error handling"""
        print("\n🧪 Testing error handling...")
        
        error_tests = [
            {
                'name': 'Invalid endpoint',
                'url': f"{self.base_url}/invalid",
                'method': 'GET',
                'expected_status': 404
            },
            {
                'name': 'Missing data',
                'url': f"{self.base_url}/predict/titanic",
                'method': 'POST',
                'data': {},
                'expected_status': 400
            }
        ]
        
        success_count = 0
        
        for test in error_tests:
            try:
                if test['method'] == 'GET':
                    response = requests.get(test['url'], timeout=5)
                else:
                    response = requests.post(
                        test['url'],
                        json=test.get('data', {}),
                        headers={'Content-Type': 'application/json'},
                        timeout=5
                    )
                
                if response.status_code == test['expected_status']:
                    print(f"   ✅ {test['name']}: Correct error handling")
                    success_count += 1
                else:
                    print(f"   ❌ {test['name']}: Expected {test['expected_status']}, got {response.status_code}")
                    
            except requests.exceptions.RequestException as e:
                print(f"   ❌ {test['name']}: {e}")
        
        return success_count == len(error_tests)
    
    def test_performance(self, num_requests=10):
        """Test API performance"""
        print(f"\n🧪 Testing performance ({num_requests} requests)...")
        
        test_data = {'pclass': 2, 'sex': 'female', 'age': 28, 'fare': 20}
        response_times = []
        success_count = 0
        
        for i in range(num_requests):
            try:
                start_time = time.time()
                response = requests.post(
                    f"{self.base_url}/predict/titanic",
                    json=test_data,
                    headers={'Content-Type': 'application/json'},
                    timeout=10
                )
                end_time = time.time()
                
                if response.status_code == 200:
                    response_times.append(end_time - start_time)
                    success_count += 1
                    
            except requests.exceptions.RequestException:
                pass
        
        if response_times:
            avg_time = sum(response_times) / len(response_times)
            min_time = min(response_times)
            max_time = max(response_times)
            
            print(f"   ✅ Performance test completed")
            print(f"   📊 Successful requests: {success_count}/{num_requests}")
            print(f"   ⏱️ Average response time: {avg_time:.3f}s")
            print(f"   ⚡ Fastest response: {min_time:.3f}s")
            print(f"   🐌 Slowest response: {max_time:.3f}s")
            
            return avg_time < 1.0  # Consider good if under 1 second
        else:
            print(f"   ❌ No successful requests")
            return False
    
    def run_all_tests(self):
        """Run comprehensive API test suite"""
        print("🧪 COMPREHENSIVE API TEST SUITE")
        print("=" * 50)
        print(f"🎯 Testing API at: {self.base_url}")
        
        tests = [
            ('Health Check', self.test_health_endpoint),
            ('Models Endpoint', self.test_models_endpoint),
            ('Titanic Prediction', self.test_titanic_prediction),
            ('Housing Prediction', self.test_housing_prediction),
            ('Error Handling', self.test_error_handling),
            ('Performance', self.test_performance)
        ]
        
        results = []
        
        for test_name, test_func in tests:
            try:
                result = test_func()
                results.append((test_name, result))
            except Exception as e:
                print(f"   ❌ {test_name} failed with exception: {e}")
                results.append((test_name, False))
        
        # Summary
        print("\n" + "=" * 50)
        print("📊 TEST RESULTS SUMMARY")
        print("=" * 50)
        
        passed = 0
        total = len(results)
        
        for test_name, result in results:
            status = "✅ PASS" if result else "❌ FAIL"
            print(f"   {test_name:<20} {status}")
            if result:
                passed += 1
        
        print(f"\n🎯 Overall: {passed}/{total} tests passed ({passed/total*100:.1f}%)")
        
        if passed == total:
            print("🎉 All tests passed! API is ready for production.")
        elif passed >= total * 0.8:
            print("⚠️ Most tests passed. Review failed tests before production.")
        else:
            print("❌ Multiple test failures. API needs fixes before production.")
        
        return passed, total

# Create API tester
api_tester = APITester()

print("🧪 API Testing Suite Ready!")
print("\n💡 To run tests:")
print("   1. Start the API server first")
print("   2. Run: api_tester.run_all_tests()")
print("\n🚀 Or test individual endpoints:")
print("   • api_tester.test_health_endpoint()")
print("   • api_tester.test_titanic_prediction()")
print("   • api_tester.test_housing_prediction()")

## 🚀 Start the API Server

Let's start our API server and test it!

In [ ]:
# Start API server in background
if FLASK_AVAILABLE and api.app:
    print("🚀 Starting API server in background...")
    
    # Start server in background thread
    server_started = api.start_server_background()
    
    if server_started:
        # Wait a moment for server to start
        time.sleep(2)
        
        # Test the server
        print("\n🧪 Running API tests...")
        passed, total = api_tester.run_all_tests()
        
        print(f"\n🌐 API Server Information:")
        print(f"   📍 Server URL: http://localhost:5000")
        print(f"   📚 API Documentation: http://localhost:5000/")
        print(f"   ❤️ Health Check: http://localhost:5000/health")
        print(f"   🤖 Models List: http://localhost:5000/models")
        print(f"   🚢 Titanic Prediction: POST http://localhost:5000/predict/titanic")
        print(f"   🏠 Housing Prediction: POST http://localhost:5000/predict/housing")
        
        print(f"\n📄 Web Interface: {Path('web_interface.html').absolute()}")
        print(f"🐳 Docker Deployment: {Path('deployment').absolute()}")
        
        # Keep server running
        print(f"\n✅ API server is running in background!")
        print(f"🛑 To stop: api.stop_server()")
        
    else:
        print("❌ Failed to start API server")
        
else:
    print("⚠️ Cannot start API server - Flask not available or API not created")
    print("\n💡 Manual testing available:")
    print("   1. Install Flask: pip install flask")
    print("   2. Run: python web_app.py (from project root)")
    print("   3. Open: http://localhost:5000")

## 📊 Deployment Monitoring and Logging

Let's implement monitoring and logging for production deployment.

In [ ]:
class DeploymentMonitor:
    """Monitor deployment health and performance"""
    
    def __init__(self, api_url="http://localhost:5000"):
        self.api_url = api_url
        self.monitoring_data = []
        self.alerts = []
    
    def check_health(self):
        """Check API health"""
        try:
            response = requests.get(f"{self.api_url}/health", timeout=5)
            if response.status_code == 200:
                data = response.json()
                return {
                    'status': 'healthy',
                    'uptime': data.get('uptime_seconds', 0),
                    'predictions': data.get('predictions_served', 0),
                    'models': data.get('models_loaded', 0),
                    'timestamp': datetime.now()
                }
            else:
                return {
                    'status': 'unhealthy',
                    'error': f'HTTP {response.status_code}',
                    'timestamp': datetime.now()
                }
        except Exception as e:
            return {
                'status': 'error',
                'error': str(e),
                'timestamp': datetime.now()
            }
    
    def monitor_performance(self, duration_minutes=5, interval_seconds=30):
        """Monitor API performance over time"""
        print(f"📊 Starting performance monitoring for {duration_minutes} minutes...")
        print(f"⏱️ Checking every {interval_seconds} seconds")
        
        start_time = time.time()
        end_time = start_time + (duration_minutes * 60)
        
        monitoring_data = []
        
        while time.time() < end_time:
            # Check health
            health_data = self.check_health()
            
            # Test response time
            response_time = self._test_response_time()
            health_data['response_time'] = response_time
            
            monitoring_data.append(health_data)
            
            # Print status
            status_emoji = "✅" if health_data['status'] == 'healthy' else "❌"
            print(f"{status_emoji} {health_data['timestamp'].strftime('%H:%M:%S')} - "
                  f"Status: {health_data['status']}, "
                  f"Response: {response_time:.3f}s, "
                  f"Predictions: {health_data.get('predictions', 0)}")
            
            # Check for alerts
            self._check_alerts(health_data)
            
            time.sleep(interval_seconds)
        
        self.monitoring_data.extend(monitoring_data)
        
        # Generate report
        self._generate_monitoring_report(monitoring_data)
        
        return monitoring_data
    
    def _test_response_time(self):
        """Test API response time"""
        try:
            start_time = time.time()
            response = requests.get(f"{self.api_url}/health", timeout=5)
            end_time = time.time()
            
            if response.status_code == 200:
                return end_time - start_time
            else:
                return -1  # Error indicator
        except:
            return -1  # Error indicator
    
    def _check_alerts(self, health_data):
        """Check for alert conditions"""
        alerts = []
        
        # Check if API is down
        if health_data['status'] != 'healthy':
            alerts.append({
                'level': 'critical',
                'message': f"API is {health_data['status']}",
                'timestamp': health_data['timestamp']
            })
        
        # Check response time
        response_time = health_data.get('response_time', 0)
        if response_time > 2.0:  # Slow response
            alerts.append({
                'level': 'warning',
                'message': f"Slow response time: {response_time:.3f}s",
                'timestamp': health_data['timestamp']
            })
        
        # Check if no models loaded
        if health_data.get('models', 0) == 0:
            alerts.append({
                'level': 'warning',
                'message': "No models loaded",
                'timestamp': health_data['timestamp']
            })
        
        # Store alerts
        self.alerts.extend(alerts)
        
        # Print alerts
        for alert in alerts:
            level_emoji = "🚨" if alert['level'] == 'critical' else "⚠️"
            print(f"   {level_emoji} ALERT: {alert['message']}")
    
    def _generate_monitoring_report(self, monitoring_data):
        """Generate monitoring report"""
        if not monitoring_data:
            return
        
        print("\n📊 MONITORING REPORT")
        print("=" * 40)
        
        # Calculate statistics
        healthy_count = sum(1 for d in monitoring_data if d['status'] == 'healthy')
        total_count = len(monitoring_data)
        uptime_percentage = (healthy_count / total_count) * 100
        
        response_times = [d['response_time'] for d in monitoring_data if d['response_time'] > 0]
        
        print(f"📈 Uptime: {uptime_percentage:.1f}% ({healthy_count}/{total_count} checks)")
        
        if response_times:
            avg_response = sum(response_times) / len(response_times)
            min_response = min(response_times)
            max_response = max(response_times)
            
            print(f"⏱️ Response Time:")
            print(f"   Average: {avg_response:.3f}s")
            print(f"   Fastest: {min_response:.3f}s")
            print(f"   Slowest: {max_response:.3f}s")
        
        # Predictions served
        predictions_data = [d.get('predictions', 0) for d in monitoring_data if d.get('predictions') is not None]
        if len(predictions_data) >= 2:
            predictions_growth = predictions_data[-1] - predictions_data[0]
            print(f"📊 Predictions served during monitoring: {predictions_growth}")
        
        # Alerts summary
        if self.alerts:
            critical_alerts = sum(1 for a in self.alerts if a['level'] == 'critical')
            warning_alerts = sum(1 for a in self.alerts if a['level'] == 'warning')
            
            print(f"🚨 Alerts:")
            print(f"   Critical: {critical_alerts}")
            print(f"   Warnings: {warning_alerts}")
        else:
            print(f"✅ No alerts during monitoring period")
        
        # Save report
        self._save_monitoring_report(monitoring_data)
    
    def _save_monitoring_report(self, monitoring_data):
        """Save monitoring report to file"""
        report_file = f"monitoring_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.json"
        
        report_data = {
            'monitoring_period': {
                'start': monitoring_data[0]['timestamp'].isoformat() if monitoring_data else None,
                'end': monitoring_data[-1]['timestamp'].isoformat() if monitoring_data else None,
                'duration_minutes': len(monitoring_data) * 0.5  # Assuming 30s intervals
            },
            'summary': {
                'total_checks': len(monitoring_data),
                'healthy_checks': sum(1 for d in monitoring_data if d['status'] == 'healthy'),
                'uptime_percentage': (sum(1 for d in monitoring_data if d['status'] == 'healthy') / len(monitoring_data)) * 100 if monitoring_data else 0
            },
            'alerts': self.alerts,
            'raw_data': [{
                'timestamp': d['timestamp'].isoformat(),
                'status': d['status'],
                'response_time': d.get('response_time', -1),
                'predictions': d.get('predictions', 0),
                'models': d.get('models', 0)
            } for d in monitoring_data]
        }
        
        try:
            with open(report_file, 'w') as f:
                json.dump(report_data, f, indent=2)
            print(f"📄 Monitoring report saved: {report_file}")
        except Exception as e:
            print(f"⚠️ Could not save monitoring report: {e}")

# Create deployment monitor
monitor = DeploymentMonitor()

print("📊 Deployment Monitor Ready!")
print("\n💡 Available monitoring functions:")
print("   • monitor.check_health() - Single health check")
print("   • monitor.monitor_performance(duration_minutes=2) - Continuous monitoring")

# Quick health check
if FLASK_AVAILABLE:
    print("\n🔍 Quick health check...")
    health = monitor.check_health()
    status_emoji = "✅" if health['status'] == 'healthy' else "❌"
    print(f"{status_emoji} API Status: {health['status']}")
    if 'error' in health:
        print(f"   Error: {health['error']}")
    else:
        print(f"   Models: {health.get('models', 0)}")
        print(f"   Predictions: {health.get('predictions', 0)}")
        print(f"   Uptime: {health.get('uptime', 0)}s")

## 📄 Generate Deployment Documentation

Let's create comprehensive deployment documentation.

In [ ]:
def generate_deployment_documentation():
    """Generate comprehensive deployment documentation"""
    print("📄 Generating deployment documentation...")
    
    doc_content = f"""
# 🚀 ML Model Deployment Guide

**Generated:** {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## 📊 Deployment Overview

This guide covers the complete deployment of our machine learning models using modern MLOps practices.

### 🎯 Deployment Components

- **REST API**: Flask-based model serving API
- **Web Interface**: User-friendly prediction interface
- **Docker Container**: Containerized deployment
- **Monitoring**: Health checks and performance monitoring
- **Documentation**: Complete API documentation

### 📈 Model Performance

| Model | Dataset | Algorithm | Performance | Status |
|-------|---------|-----------|-------------|--------|
| Titanic Classifier | Titanic | Logistic Regression | 89.4% Accuracy | ✅ Production |
| Housing Regressor | Boston Housing | Linear Regression | R² = 0.681 | ✅ Production |

## 🌐 API Endpoints

### Base URL
```
http://localhost:5000
```

### Available Endpoints

#### 1. Health Check
```http
GET /health
```

**Response:**
```json
{{
  "status": "healthy",
  "uptime_seconds": 3600,
  "models_loaded": 2,
  "predictions_served": 150,
  "timestamp": "2024-01-15T10:30:00"
}}
```

#### 2. List Models
```http
GET /models
```

**Response:**
```json
{{
  "models": [
    {{
      "model_id": "titanic_logistic_regression",
      "algorithm": "Logistic Regression",
      "dataset": "titanic",
      "task_type": "classification",
      "performance": 0.894
    }}
  ],
  "total_models": 2
}}
```

#### 3. Titanic Survival Prediction
```http
POST /predict/titanic
Content-Type: application/json
```

**Request Body:**
```json
{{
  "pclass": 1,
  "sex": "female",
  "age": 25,
  "fare": 50.0
}}
```

**Response:**
```json
{{
  "prediction": 1,
  "survival_probability": 0.85,
  "result": "Survived",
  "model": "Logistic Regression",
  "timestamp": "2024-01-15T10:30:00"
}}
```

#### 4. Housing Price Prediction
```http
POST /predict/housing
Content-Type: application/json
```

**Request Body:**
```json
{{
  "crime_rate": 0.1,
  "rooms": 6.5,
  "age": 30,
  "distance": 4.0
}}
```

**Response:**
```json
{{
  "predicted_price": 25.3,
  "price_range": "Medium",
  "model": "Linear Regression",
  "timestamp": "2024-01-15T10:30:00"
}}
```

## 🚀 Deployment Options

### Option 1: Local Development

```bash
# Install dependencies
pip install -r requirements.txt

# Start the API server
python web_app.py

# Access the API
curl http://localhost:5000/health
```

### Option 2: Docker Deployment

```bash
# Navigate to deployment directory
cd deployment/

# Build and start containers
docker-compose up --build

# Access the application
# Web Interface: http://localhost
# API: http://localhost/api/health
```

### Option 3: Production Deployment

```bash
# Using Gunicorn for production
gunicorn --bind 0.0.0.0:5000 --workers 4 app:app

# With Nginx reverse proxy
# See deployment/nginx.conf for configuration
```

## 🐳 Docker Configuration

### Dockerfile
```dockerfile
FROM python:3.9-slim
WORKDIR /app
COPY requirements.txt .
RUN pip install -r requirements.txt
COPY . .
EXPOSE 5000
CMD ["python", "app.py"]
```

### Docker Compose
```yaml
version: '3.8'
services:
  ml-api:
    build: .
    ports:
      - "5000:5000"
    volumes:
      - ./models:/app/models:ro
```

## 📊 Monitoring and Logging

### Health Monitoring

The API includes comprehensive health monitoring:

- **Health Endpoint**: `/health` - Real-time status
- **Metrics Tracking**: Request count, response times
- **Model Status**: Loaded models and performance
- **Uptime Monitoring**: Service availability

### Logging

All API requests and responses are logged:

```python
# Log format
2024-01-15 10:30:00 - INFO - Prediction made with titanic_model: 1
2024-01-15 10:30:01 - ERROR - Prediction error: Invalid input format
```

### Performance Monitoring

```python
# Monitor API performance
monitor = DeploymentMonitor()
monitor.monitor_performance(duration_minutes=5)
```

## 🧪 Testing

### Automated Testing

```python
# Run comprehensive API tests
api_tester = APITester()
passed, total = api_tester.run_all_tests()
```

### Manual Testing

```bash
# Test health endpoint
curl http://localhost:5000/health

# Test Titanic prediction
curl -X POST http://localhost:5000/predict/titanic \
  -H "Content-Type: application/json" \
  -d '{"pclass": 1, "sex": "female", "age": 25, "fare": 50}'

# Test Housing prediction
curl -X POST http://localhost:5000/predict/housing \
  -H "Content-Type: application/json" \
  -d '{"crime_rate": 0.1, "rooms": 6.5, "age": 30, "distance": 4}'
```

## 🔒 Security Considerations

### Production Security

1. **HTTPS**: Use SSL/TLS certificates
2. **Authentication**: Implement API key authentication
3. **Rate Limiting**: Prevent API abuse
4. **Input Validation**: Validate all input data
5. **CORS**: Configure Cross-Origin Resource Sharing

### Example Security Headers

```python
@app.after_request
def after_request(response):
    response.headers['X-Content-Type-Options'] = 'nosniff'
    response.headers['X-Frame-Options'] = 'DENY'
    response.headers['X-XSS-Protection'] = '1; mode=block'
    return response
```

## 📈 Scaling Considerations

### Horizontal Scaling

```yaml
# Docker Compose scaling
docker-compose up --scale ml-api=3
```

### Load Balancing

```nginx
upstream ml_api {{
    server ml-api-1:5000;
    server ml-api-2:5000;
    server ml-api-3:5000;
}}
```

### Performance Optimization

1. **Model Caching**: Keep models in memory
2. **Connection Pooling**: Reuse database connections
3. **Async Processing**: Use async frameworks for high throughput
4. **CDN**: Use Content Delivery Network for static assets

## 🚨 Troubleshooting

### Common Issues

#### API Server Won't Start
```bash
# Check if port is in use
lsof -i :5000  # Mac/Linux
netstat -ano | findstr :5000  # Windows

# Use different port
export PORT=5001
python app.py
```

#### Models Not Loading
```bash
# Check model files exist
ls -la trained_models/

# Check file permissions
chmod 644 trained_models/*.joblib
```

#### Prediction Errors
```python
# Check input format
# Ensure all required fields are present
# Validate data types (numbers vs strings)
```

### Debug Mode

```python
# Enable debug mode for development
app.run(debug=True)
```

## 📞 Support

### Monitoring Alerts

Set up alerts for:
- API downtime
- High response times (>2 seconds)
- Error rates (>5%)
- Model loading failures

### Maintenance

Regular maintenance tasks:
- Update model versions
- Monitor disk space
- Review logs for errors
- Update dependencies

## 🎯 Next Steps

1. **Model Updates**: Implement automated model retraining
2. **A/B Testing**: Compare model versions in production
3. **Advanced Monitoring**: Add custom metrics and dashboards
4. **CI/CD Pipeline**: Automate deployment process
5. **Cloud Deployment**: Deploy to AWS, GCP, or Azure

---

**📄 Documentation Version**: 1.0  
**🔄 Last Updated**: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}  
**👨‍💻 Generated by**: ML Pipeline Tutorial Series
    """.strip()
    
    # Save documentation
    doc_file = "deployment_guide.md"
    with open(doc_file, 'w', encoding='utf-8') as f:
        f.write(doc_content)
    
    print(f"✅ Deployment documentation created: {doc_file}")
    
    # Create quick reference
    quick_ref = f"""
# 🚀 Quick Deployment Reference

## Start API Server
```bash
python web_app.py
```

## Test API
```bash
curl http://localhost:5000/health
```

## Docker Deployment
```bash
cd deployment/
docker-compose up --build
```

## Key URLs
- Web Interface: http://localhost:5000/
- Health Check: http://localhost:5000/health
- API Docs: http://localhost:5000/

## Test Predictions
```bash
# Titanic
curl -X POST http://localhost:5000/predict/titanic \
  -H "Content-Type: application/json" \
  -d '{"pclass": 1, "sex": "female", "age": 25, "fare": 50}'

# Housing
curl -X POST http://localhost:5000/predict/housing \
  -H "Content-Type: application/json" \
  -d '{"crime_rate": 0.1, "rooms": 6.5, "age": 30, "distance": 4}'
```
    """.strip()
    
    quick_ref_file = "quick_deployment_reference.md"
    with open(quick_ref_file, 'w', encoding='utf-8') as f:
        f.write(quick_ref)
    
    print(f"✅ Quick reference created: {quick_ref_file}")
    
    return doc_file, quick_ref_file

# Generate documentation
doc_file, quick_ref_file = generate_deployment_documentation()
print("\n📚 Deployment documentation ready!")

## 🎉 Congratulations!

You've successfully completed the entire ML Pipeline tutorial series! You now have a **production-ready machine learning deployment** with:

✅ **REST API**: Professional model serving endpoints  
✅ **Web Interface**: User-friendly prediction interface  
✅ **Docker Deployment**: Containerized production deployment  
✅ **Monitoring**: Health checks and performance monitoring  
✅ **Testing**: Comprehensive API testing suite  
✅ **Documentation**: Complete deployment guides  

### 🏆 Complete MLOps Pipeline Achieved

**🔄 End-to-End Pipeline:**
1. **Data Exploration** → Understanding datasets and patterns
2. **Feature Engineering** → Creating powerful predictive features
3. **Model Training** → Training and optimizing multiple algorithms
4. **Experiment Tracking** → Professional MLflow experiment management
5. **Model Deployment** → **← COMPLETED** → Production-ready serving

### 🚀 What You've Built

**🌐 Production API:**
- **REST Endpoints**: `/health`, `/models`, `/predict/titanic`, `/predict/housing`
- **Performance**: Sub-second response times with comprehensive error handling
- **Monitoring**: Real-time health checks and usage statistics
- **Documentation**: Interactive API documentation and examples

**🖥️ Web Interface:**
- **Modern UI**: Responsive design with professional styling
- **Real-time Predictions**: Instant model predictions with probability scores
- **Error Handling**: Graceful error messages and loading states
- **Cross-platform**: Works on desktop, tablet, and mobile

**🐳 Docker Deployment:**
- **Multi-container Setup**: API server + Nginx reverse proxy
- **Production Ready**: Health checks, logging, and monitoring
- **Scalable**: Easy horizontal scaling with load balancing
- **Secure**: Non-root user, security headers, and best practices

### 🔧 Technical Achievements

1. **Professional API Design**: RESTful endpoints with proper HTTP status codes
2. **Error Handling**: Comprehensive exception handling and user feedback
3. **Performance Optimization**: Efficient model loading and caching
4. **Monitoring Integration**: Real-time health and performance monitoring
5. **Testing Framework**: Automated testing suite with performance benchmarks
6. **Production Deployment**: Docker containerization with Nginx proxy

### 📁 Files Created

Your complete deployment package includes:
- **`web_interface.html`** - Modern web interface
- **`deployment/`** - Complete Docker deployment configuration
- **`deployment_guide.md`** - Comprehensive deployment documentation
- **`quick_deployment_reference.md`** - Quick start guide
- **API Server** - Production-ready Flask application
- **Monitoring Tools** - Performance and health monitoring

### 🎯 Ready for Production

Your ML models are now ready for:
- **Production Deployment**: Docker containers with proper monitoring
- **Team Collaboration**: Complete documentation and testing
- **Scaling**: Horizontal scaling with load balancing
- **Maintenance**: Health monitoring and automated alerts

### 💡 Next Level Enhancements

Take your deployment further with:
1. **Cloud Deployment**: AWS, GCP, or Azure deployment
2. **CI/CD Pipeline**: Automated testing and deployment
3. **Advanced Monitoring**: Custom dashboards and alerting
4. **A/B Testing**: Compare model versions in production
5. **Auto-scaling**: Dynamic scaling based on load
6. **Model Updates**: Automated retraining and deployment

### 🚀 Quick Start Commands

**Start your production API:**
```bash
# Local development
python web_app.py

# Docker production
cd deployment/
docker-compose up --build
```

**Access your deployment:**
- 🌐 **Web Interface**: http://localhost:5000/
- ❤️ **Health Check**: http://localhost:5000/health
- 📚 **API Docs**: http://localhost:5000/
- 🤖 **Models**: http://localhost:5000/models

### 🏆 Professional MLOps Achievement

You've successfully implemented a **complete MLOps pipeline** that includes:
- ✅ Data versioning and experiment tracking
- ✅ Model training and hyperparameter optimization
- ✅ Model registry and lifecycle management
- ✅ Production deployment with monitoring
- ✅ Comprehensive testing and documentation

**🎊 Congratulations on completing the entire ML Pipeline tutorial series!**

You now have the skills and tools to build production-ready machine learning systems that can scale and serve real users. Your models are deployed, monitored, and ready for the world! 🌟

---

**🎯 Your ML Pipeline is Production Ready!**  
**🚀 Time to serve real predictions to real users!**